# 1. Titanic
* 출력을 csv 파일로 저장하여 제출! (Jupyter notebook code도 함께 제출)
* Remember our goal: we want to find patterns in train.csv that help us predict whether the passengers in test.csv survived.

In [1]:
import pandas as pd

train = pd.read_csv("C:/Users/yune4/titanic/train.csv")
test = pd.read_csv("C:/Users/yune4/titanic/test.csv")
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* PassengerId - 탑승자 번호  (합습에 이용 안함)
* Survived - 생존 여부 (0 = 사망, 1 = 생존)  (학습에 이용)
* Pclass - 티켓 클래스 (1 = 1등석, 2 = 2등석, 3 = 3등석)  (학습에 이용)
* Sex - 성별  (학습에 이용)
* Age - 나이  (학습에 이용)
* SibSp - 함께 탑승한 자녀 / 배우자 의 수     ----┐합하여 Family로 (학습에 이용)
* Parch - 함께 탑승한 부모님 / 아이들 의 수   -┘
* Ticket - 티켓 번호  (학습에 이용 안함)
* Fare - 탑승 요금    (학습에 이용)
* Cabin - 수하물 번호  (학습에 이용)
* Embarked - 선착장 (C = Cherbourg, Q = Queenstown, S = Southampton)  (학습에 이용)

In [2]:
print(train.isna().sum(axis=0)) #age와 cabin, embarked에 결측값 있음

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [3]:
train['Name'] = train['Name'].apply(lambda x: x.split(',')[1].strip())
train['Title'] = train['Name'].apply(lambda x: x.split('.')[0].strip())
train['Title'] #후에 age결측값 처리 위해 미리 만들어둠

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886     Rev
887    Miss
888    Miss
889      Mr
890      Mr
Name: Title, Length: 891, dtype: object

In [4]:
train_df = train.drop(['PassengerId', 'Ticket',  'Name'], axis=1, inplace=False) #안쓰는 열 삭제
train_df['Family'] = train_df['SibSp'] + train_df['Parch'] #두 데이터 합하여 하나의 데이터로
train_df.drop(['SibSp', 'Parch'], axis=1, inplace=True) #원래의 열 삭제

In [5]:
gender_mapping = {'female': 1, 'male': 0} #성별 숫자로
train_df['Sex'] = train_df['Sex'].map(gender_mapping) 

In [6]:
embarked_mapping = {'S': 2, 'Q': 1, 'C': 0} #항구 숫자로
train_df['Embarked'] = train_df['Embarked'].map(embarked_mapping) 
train_df['Embarked'] = train_df['Embarked'].fillna(train_df['Embarked'].mode()[0]) #항구 결측치를 최빈값으로 처리

In [7]:
train_df['Cabin'] = train_df['Cabin'].apply(lambda x: x[:1] if type(x)==str else str(x)) #알파벳만 가져옴

#cabin mapping
i = 0  
cabin_mapping = {}
for idx in train_df['Cabin'].value_counts().index:
    cabin_mapping[idx] = i
    i += 1
    
print(cabin_mapping)

train_df['Cabin'] = train_df['Cabin'].map(cabin_mapping)

{'nan': 0, 'C': 1, 'B': 2, 'D': 3, 'E': 4, 'A': 5, 'F': 6, 'G': 7, 'T': 8}


In [8]:
train_df['Age'] = train_df['Age'].fillna(train_df.groupby('Title')['Age'].transform('mean')) #호칭별 age 평균값 이용하여 age결측값 처리
print(train_df.isna().sum(axis=0)) #결측값 처리된 것 보여줌
train_df.drop(['Title'], axis=1, inplace=True) #Title 열 삭제

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
Cabin       0
Embarked    0
Title       0
Family      0
dtype: int64


In [9]:
train_df.head() #최종 train dataset 완성

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Family
0,0,3,0,22.0,7.2500,0,2.0,1
1,1,1,1,38.0,71.2833,1,0.0,1
2,1,3,1,26.0,7.9250,0,2.0,0
3,1,1,1,35.0,53.1000,1,2.0,1
4,0,3,0,35.0,8.0500,0,2.0,0


위의 train data와 같은 방법으로 test data 처리

In [10]:
print(test.isna().sum(axis=0))

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [11]:
test['Name'] = test['Name'].apply(lambda x: x.split(',')[1].strip())
test['Title'] = test['Name'].apply(lambda x: x.split('.')[0].strip())
test['Title']

0          Mr
1         Mrs
2          Mr
3          Mr
4         Mrs
        ...  
413        Mr
414      Dona
415        Mr
416        Mr
417    Master
Name: Title, Length: 418, dtype: object

In [12]:
test_df = test.drop(['PassengerId', 'Ticket',  'Name'], axis=1, inplace=False)
test_df['Family'] = test_df['SibSp'] + test_df['Parch'] 
test_df.drop(['SibSp', 'Parch'], axis=1, inplace=True)
test_df['Sex'] = test_df['Sex'].map(gender_mapping)
test_df['Embarked'] = test_df['Embarked'].map(embarked_mapping) 

In [13]:
test_df['Cabin'] = test_df['Cabin'].apply(lambda x: x[:1] if type(x)==str else str(x))
test_df['Cabin'] = test_df['Cabin'].map(cabin_mapping)

In [14]:
test_df['Age'] = test_df['Age'].fillna(test_df.groupby('Title')['Age'].transform('mean'))
print(test_df.isna().sum(axis=0)) 
test_df.drop(['Title'], axis=1, inplace=True) 

Pclass      0
Sex         0
Age         1
Fare        1
Cabin       0
Embarked    0
Title       0
Family      0
dtype: int64


In [15]:
#위에서 결측치 처리가 안된 값은 각 컬럼의 평균값으로 대체
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].mean())
print(test_df.isna().sum(axis=0)) 

Pclass      0
Sex         0
Age         0
Fare        0
Cabin       0
Embarked    0
Family      0
dtype: int64


In [16]:
test_df.head() #최종 test dataset 완성

,Pclass,Sex,Age,Fare,Cabin,Embarked,Family
0,3,0,34.5,7.8292,0,1,0
1,3,1,47.0,7.0000,0,2,1
2,2,0,62.0,9.6875,0,1,0
3,3,0,27.0,8.6625,0,2,0
4,3,1,22.0,12.2875,0,2,2


In [21]:
from sklearn.ensemble import RandomForestClassifier  #예측 후 저장

y = train_df["Survived"]

features = ["Pclass", "Sex", "Family", "Age", "Embarked", "Fare", "Cabin"]
X = pd.get_dummies(train_df[features])
X_test = pd.get_dummies(test_df[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
